In [1]:
#Importing Required Libraries
!pip install --upgrade google-api-python-client --quiet
import json
import googleapiclient
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd

In [2]:
import json

path_to_json = "info.json"

with open(path_to_json, "r") as handler:
    info = json.load(handler)

API_KEY = info["API_KEY"]
!pip install --upgrade google-api-python-client --quiet

In [3]:
youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=API_KEY)

## In this section, we collect data from a wide variety of the channels. The output of this would be a dataframe with the following structure - 

<!-- ##### { 
'channel_name' : ,
'video_id' : ,
'video_like' :,
'video_comments' :
'video_category' :
} -->

In [4]:
# Video & Metrics Calculation
import datetime
import pandas as pd
from googleapiclient.discovery import build

def download_statistics(username, limit=10, api_key=None):
    
    request = youtube.channels().list(
    part="contentDetails",
    forUsername=username
    )
    
    res = request.execute()
    uploads_playlist_id = res["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]
    videos_info = []
    
    request = youtube.playlistItems().list(
        part="snippet",
        playlistId = uploads_playlist_id
        # YOUR SOLUTION
    )
    
    res = request.execute()
    
    page_token = res['nextPageToken'] if res['nextPageToken'] else ""
    for v in res["items"]:
        video_id = v['snippet']['resourceId']['videoId']
            
        # Get video views count
        stats_request = youtube.videos().list(
            part="statistics",
            id=video_id
        )
        stats_res = stats_request.execute()
            
        # Check if statistics are available for the video
        if stats_res['items']:
            statistics = stats_res['items'][0]['statistics']
            view_count = int(statistics.get('viewCount', 0))
            like_count = int(statistics.get('likeCount',0))
            comment_count = int(statistics.get('commentCount',0))
        else:
            view_count = like_count = comment_count = 0
    
        dict_vid = {
            'channel_name': v['snippet']['channelTitle'],
            'video_id': video_id,
            'video_title': v['snippet']['title'],
            'video_description': v['snippet']['description'],
            'video_published_at': v['snippet']['publishedAt'],
            'video_view_count': view_count,
            'video_like_count': like_count,
            'video_comment_count': comment_count
        }
        videos_info.append(dict_vid)

    while page_token and len(videos_info) < limit: # Limiting the number of API calls using the videos_info as the daily limit is exceeding otherwise
        request = youtube.playlistItems().list(
        part="snippet",
        playlistId = uploads_playlist_id,
        pageToken = page_token
    )
        res = request.execute()
        page_token = res['nextPageToken'] if res['nextPageToken'] else ""
    
        for v in res["items"]:
            video_id = v['snippet']['resourceId']['videoId']
            
            # Get video video views count
            stats_request = youtube.videos().list(
                part="statistics",
                id=video_id
            )
            stats_res = stats_request.execute()
            
            # Check if statistics are available for the video
            if stats_res['items']:
                statistics = stats_res['items'][0]['statistics']
                view_count = int(statistics.get('viewCount', 0))
                like_count = int(statistics.get('likeCount',0))
                comment_count = int(statistics.get('commentCount',0))
            else:
                view_count = like_count = comment_count = 0
    
            dict_vid = {
                'channel_name': v['snippet']['channelTitle'],
                'video_id': video_id,
                'video_title': v['snippet']['title'],
                'video_description': v['snippet']['description'],
                'video_published_at': v['snippet']['publishedAt'],
                'video_view_count': view_count,
                'video_like_count': like_count,
                'video_comment_count': comment_count
            }
            videos_info.append(dict_vid)
    print(f"The videos extracted from {username} are: ", len(videos_info))
    return videos_info
    

API_KEY = API_KEY
youtube_channel_1 = [
    "Vsauce",
    "CNN",
    "moneycontrol",   
]

export_data = []
for channel in youtube_channel_1:
    export_data.extend(download_statistics(channel, limit=5, api_key=API_KEY))

current_datetime = datetime.datetime.now()
transcript_data_path = f"data/transcript_data_{str(current_datetime)}.csv"
#print('The number of total extracted videos is:', len(export_data))
#print(json.dumps(export_data,indent=2))
pd.DataFrame(export_data).to_csv(transcript_data_path)


The videos extracted from Vsauce are:  5
The videos extracted from CNN are:  5
The videos extracted from moneycontrol are:  5


In [5]:
!pip install youtube-transcript-api --quiet
from youtube_transcript_api import YouTubeTranscriptApi

In [6]:
# Transcript download

transcript_data = pd.read_csv(transcript_data_path)

# Function to get transcript for a given video_id
def get_transcript(video_id):
    print(video_id)
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return ' '.join([entry['text'] for entry in transcript])
    except Exception as e:
        # print(e)
        return ''

# Apply the function to create a new 'transcript' column
transcript_data['transcript'] = transcript_data['video_id'].apply(get_transcript)
transcript_data.to_csv(transcript_data_path)
transcript_data.head()

    
    


ynGb9FcH5ME
pnt-GwDokts
r734u7g80Zw
UuqBFnhoqhM
F9QS_qbeNLU
iWsFdmCQV74
k6iYXVvKxoI
UuY7PMpYjLE
prsWw4q8XOM
3HGwjgYQgCU
hQyPOJ641vQ
ADO095Z3V7M
H3XoUT7tLoI
ePiTTJ-8ULc
KI0VAK-29fE


,Unnamed: 0,channel_name,video_id,video_title,video_description,video_published_at,video_view_count,video_like_count,video_comment_count,transcript
0,0,Vsauce,ynGb9FcH5ME,"Okay, Here's The 'Reindeer'",NaN,2024-01-01T14:41:41Z,4262571,364254,5338,I will show you the reindeer it's pretty simpl...
1,1,Vsauce,pnt-GwDokts,The Pioneer Puzzle,NaN,2023-12-30T14:28:54Z,3702468,354024,3218,the Wonder block or Pioneer puzzle a set of do...
2,2,Vsauce,r734u7g80Zw,It's Time To Rename Uranus,NaN,2023-12-26T00:04:32Z,3936739,489296,14291,Merry Christmas it's time to rename Uranus the...
3,3,Vsauce,UuqBFnhoqhM,THE COMPLETE LIST,NaN,2023-12-22T19:44:05Z,973738,66307,794,okay I've got some exciting news about the Vsa...
4,4,Vsauce,F9QS_qbeNLU,The Most-Photographed Toilet In New Zealand,NaN,2023-12-21T01:46:48Z,2800996,224193,3082,come with me as I poop in New Zealand's most p...


In [9]:
# Content Categorization
# https://cloud.google.com/natural-language/docs/classifying-text#language-classify-content-python